In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test
from Models.basic_models import average_model, two_layer_basic, post_process

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type)
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_basic()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

using device: cuda


In [3]:
# If multiple GPU
model = nn.DataParallel(model)

In [ ]:
learning_rate = 3e-3
momentum = 0.95
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg, nesterov=True)

In [ ]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         epochs=3, print_every=30, print_level=3, lr_decay=1)


****Epoch 0 Iteration 0, loss = 2.5627, corrected loss = 1143.8507
Validation loss 1122.8934 over 81 frames
module.conv_a1.weight,   	norm: 3.8570e+00, 	update norm: 1.5209e-02 	Update/norm: 3.9431e-03
module.conv_a1.bias,   	norm: 3.0593e-01, 	update norm: 3.0970e-03 	Update/norm: 1.0123e-02
module.conv_b1.weight,   	norm: 3.9936e+00, 	update norm: 1.0353e-02 	Update/norm: 2.5923e-03
module.conv_b1.bias,   	norm: 3.5212e-01, 	update norm: 1.6068e-03 	Update/norm: 4.5631e-03
module.final.weight,   	norm: 1.3503e+00, 	update norm: 1.6212e-02 	Update/norm: 1.2007e-02
module.final.bias,   	norm: 1.5182e-01, 	update norm: 1.0908e-03 	Update/norm: 7.1846e-03

Iter 0... ... ... 
Iter 10... ... ... 
Iter 20... ... ... 
Iteration 30, loss = 0.2925, corrected loss = 140.2896
Validation loss 138.4111 over 81 frames
module.conv_a1.weight,   	norm: 3.7949e+00, 	update norm: 1.6713e-02 	Update/norm: 4.4040e-03
module.conv_a1.bias,   	norm: 4.8380e-01, 	update norm: 5.9902e-03 	Update/norm: 1.2382e

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)